Combining Province Table with b3a_tk3 (employment data) from IFLS5 Dataset

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/Users/sanhorn/Desktop/Internships/Econ Data Lab/IFLS5 Dataset/b3a_tk3.csv")
tmp_df = df.copy()
selected_columns = ["hhid14_9", "pid14", "tk28year", "tk28", "tk31a"]
tmp_df = tmp_df[selected_columns]
tmp_df["person"] = tmp_df["hhid14_9"].astype(str) + tmp_df["pid14"].astype(str)
tmp_df = tmp_df.drop(columns = ["hhid14_9", "pid14"])
tmp_df['person'] = tmp_df['person'].astype(str)

# tk28 indicates has job / no job
# tk31a indicates industry sector

province = pd.read_csv("/Users/sanhorn/Desktop/Internships/Econ Data Lab/province_unemploymentrate/transformed_province_data.csv")
province_long = pd.melt(
    province,
    id_vars=['hhid_pid'],
    value_vars=[str(year) for year in range(2007, 2015)],
    var_name='year',
    value_name='province'
)
province_long['year'] = province_long['year'].astype(int)
province_long['hhid_pid'] = province_long['hhid_pid'].astype(str)

In [3]:
result_df = tmp_df.merge(
    province_long,
    left_on=['person', 'tk28year'],
    right_on=['hhid_pid', 'year'],
    how='inner'  
)

result_df = result_df.drop(columns=['hhid_pid', 'year'])
result_df


,tk28year,tk28,tk31a,person,province
0,2007,1,1.0,10600001,12.0
1,2008,1,1.0,10600001,12.0
2,2009,1,1.0,10600001,12.0
3,2010,3,NaN,10600001,12.0
4,2011,3,NaN,10600001,12.0
...,...,...,...,...,...
69355,2010,3,NaN,32128000012,NaN
69356,2011,1,9.0,32128000012,NaN
69357,2012,1,9.0,32128000012,NaN
69358,2013,1,9.0,32128000012,NaN


In [4]:
province_to_geo_unit = {
    # Sumatra
    11: 'Sumatra',
    12: 'Sumatra',
    13: 'Sumatra',
    14: 'Sumatra',
    15: 'Sumatra',
    16: 'Sumatra',
    17: 'Sumatra',
    18: 'Sumatra',
    
    # Java
    31: 'Java',
    32: 'Java',
    33: 'Java',
    34: 'Java',
    35: 'Java',
    
    # Nusa Tenggara (Lesser Sunda Islands)
    51: 'Nusa Tenggara (Lesser Sunda Islands)',
    52: 'Nusa Tenggara (Lesser Sunda Islands)',
    53: 'Nusa Tenggara (Lesser Sunda Islands)',
    54: 'Nusa Tenggara (Lesser Sunda Islands)',
    
    # Kalimantan
    61: 'Kalimantan',
    62: 'Kalimantan',
    63: 'Kalimantan',
    64: 'Kalimantan',
    
    # Sulawesi
    71: 'Sulawesi',
    72: 'Sulawesi',
    73: 'Sulawesi',
    74: 'Sulawesi',
    
    # Maluku Islands
    81: 'Maluku Islands',
    
    # Papua (Western New Guinea)
    82: 'Papua (Western New Guinea)',
}

In [5]:
# TODO
result_df_copy = result_df.copy()
result_df_copy['province_numeric'] = pd.to_numeric(result_df_copy['province'], errors='coerce')

result_df_copy['geo_unit'] = result_df_copy['province_numeric'].map(province_to_geo_unit)
result_df_copy['geo_unit'] = result_df_copy['geo_unit'].fillna('Unknown')

# Replace the original province column with the geo_unit column
result_df_copy = result_df_copy.drop(columns=['province', 'province_numeric'])
result_df_copy


,tk28year,tk28,tk31a,person,geo_unit
0,2007,1,1.0,10600001,Sumatra
1,2008,1,1.0,10600001,Sumatra
2,2009,1,1.0,10600001,Sumatra
3,2010,3,NaN,10600001,Sumatra
4,2011,3,NaN,10600001,Sumatra
...,...,...,...,...,...
69355,2010,3,NaN,32128000012,Unknown
69356,2011,1,9.0,32128000012,Unknown
69357,2012,1,9.0,32128000012,Unknown
69358,2013,1,9.0,32128000012,Unknown


Get DataFrame for each Geographical Unit

In [6]:
df_sumatra = result_df_copy[result_df_copy['geo_unit'] == 'Sumatra']
df_java = result_df_copy[result_df_copy['geo_unit'] == 'Java']
df_nusa_tenggara = result_df_copy[result_df_copy['geo_unit'] == 'Nusa Tenggara (Lesser Sunda Islands)']
df_kalimantan = result_df_copy[result_df_copy['geo_unit'] == 'Kalimantan']
df_sulawesi = result_df_copy[result_df_copy['geo_unit'] == 'Sulawesi']
df_maluku = result_df_copy[result_df_copy['geo_unit'] == 'Maluku Islands']
df_papua = result_df_copy[result_df_copy['geo_unit'] == 'Papua (Western New Guinea)']
df_unknown = result_df_copy[result_df_copy['geo_unit'] == 'Unknown']

In [7]:
industry_sector_mapping = {
    1: "Primary",
    2: "Primary",
    3: "Secondary",
    4: "Secondary",
    5: "Secondary",
    6: "Tertiary",
    7: "Tertiary",
    8: "Tertiary",
    9: "Tertiary",
    10: "Activities Not Classified"
}

In [8]:
def sector_analysis(df): 
    df1 = df.copy()
    years = ['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014']
    sector_dfs = {}
    for i in range(1, 10):
        sector_dfs[i] = pd.DataFrame(columns=years)

    no_job_df = pd.DataFrame(columns=years)

    for _, row in df1.iterrows():
        year = str(row["tk28year"])
        person_id = str(row["person"])
        person_hasJob = row["tk28"]

        if person_hasJob == 1:
            sector = int(row["tk31a"])

            if sector in sector_dfs:
                if year in sector_dfs[sector].columns:
                    if sector_dfs[sector].empty:
                        sector_dfs[sector].loc[0] = [[] for _ in range(len(years))]
                    curr = sector_dfs[sector].at[0, year]
                    if isinstance(curr, list): curr.append(person_id)
                    else: curr = [person_id]

                    sector_dfs[sector].at[0, year] = curr

        if person_hasJob != 1:
            if no_job_df.empty:
                no_job_df.loc[0] = [[] for _ in range(len(years))]
            
            curr = no_job_df.at[0, year]
            if isinstance(curr, list): curr.append(person_id)
            else: curr = [person_id]

            no_job_df.at[0, year] = curr
        
    table_data = {year: [] for year in years}
    sector_names = [industry_sector_mapping[i] for i in range(1, 10)] + ["No Job"]

    # Populate the table with lengths of lists in sector_dfs
    for i in range(1, 10):
        for year in years:
            if not sector_dfs[i].empty and year in sector_dfs[i].columns:
                table_data[year].append(len(sector_dfs[i].at[0, year]))
            else:
                table_data[year].append(0)

    # Add lengths of lists in no_job_df
    for year in years:
        if not no_job_df.empty and year in no_job_df.columns:
            table_data[year].append(len(no_job_df.at[0, year]))
        else:
            table_data[year].append(0)

    # Create the DataFrame
    result_table = pd.DataFrame(table_data, index=sector_names)
    result_table = result_table.groupby(result_table.index).sum().reset_index()
    return result_table

In [9]:
def calculate_employment_rates(df):
    result = df.copy()
    # Get only the numeric columns (years)
    numeric_cols = [col for col in df.columns if col != 'index']
    for col in numeric_cols:
        result[col] = result[col].astype(float)
    
    # Calculate column totals
    year_totals = df[numeric_cols].sum(axis=0)
    
    # Calculate the ratios
    for index, row in df.iterrows():
        for year in numeric_cols:
            result.at[index, year] = row[year] / year_totals[year]
    return result

Industry Sectors Employment/Unemployment Rate for Each Geographical Unit

In [10]:
df_sumatra_sector = sector_analysis(df_sumatra)
ratio_sumatra = calculate_employment_rates(df_sumatra_sector)  
ratio_sumatra = ratio_sumatra.set_index('index')

df_java_sector = sector_analysis(df_java)
ratio_java = calculate_employment_rates(df_java_sector)
ratio_java = ratio_java.set_index('index')

df_nusa_tenggara_sector = sector_analysis(df_nusa_tenggara)
ratio_nusa_tenggara = calculate_employment_rates(df_nusa_tenggara_sector)
ratio_nusa_tenggara = ratio_nusa_tenggara.set_index('index')

df_kalimantan_sector = sector_analysis(df_kalimantan)
ratio_kalimantan = calculate_employment_rates(df_kalimantan_sector)
ratio_kalimantan = ratio_kalimantan.set_index('index')

df_sulawesi_sector = sector_analysis(df_sulawesi) 
ratio_sulawesi = calculate_employment_rates(df_sulawesi_sector)
ratio_sulawesi = ratio_sulawesi.set_index('index')

df_maluku_sector = sector_analysis(df_maluku)
ratio_maluku = calculate_employment_rates(df_maluku_sector)
ratio_maluku = ratio_maluku.set_index('index')

df_papua_sector = sector_analysis(df_papua)
ratio_papua = calculate_employment_rates(df_papua_sector)
ratio_papua = ratio_papua.set_index('index')

df_unknown_sector = sector_analysis(df_unknown)
ratio_unknown = calculate_employment_rates(df_unknown_sector)
ratio_unknown = ratio_unknown.set_index('index')     

/var/folders/p4/7cr7wny13_q7l7hs6wj6tlqc0000gq/T/ipykernel_33311/1076171881.py:14: RuntimeWarning: invalid value encountered in scalar divide
  result.at[index, year] = row[year] / year_totals[year]


In [11]:
ratio_sumatra.to_csv("/Users/sanhorn/Desktop/Internships/Econ Data Lab/geounits_sectors_rate/ratio_sumatra.csv")
ratio_java.to_csv("/Users/sanhorn/Desktop/Internships/Econ Data Lab/geounits_sectors_rate/ratio_java.csv")
ratio_nusa_tenggara.to_csv("/Users/sanhorn/Desktop/Internships/Econ Data Lab/geounits_sectors_rate/ratio_nusa_tenggara.csv")
ratio_kalimantan.to_csv("/Users/sanhorn/Desktop/Internships/Econ Data Lab/geounits_sectors_rate/ratio_kalimantan.csv")
ratio_sulawesi.to_csv("/Users/sanhorn/Desktop/Internships/Econ Data Lab/geounits_sectors_rate/ratio_sulawesi.csv")
ratio_maluku.to_csv("/Users/sanhorn/Desktop/Internships/Econ Data Lab/geounits_sectors_rate/ratio_maluku.csv")
ratio_papua.to_csv("/Users/sanhorn/Desktop/Internships/Econ Data Lab/geounits_sectors_rate/ratio_papua.csv")
ratio_unknown.to_csv("/Users/sanhorn/Desktop/Internships/Econ Data Lab/geounits_sectors_rate/ratio_unknown.csv")